# Job Sequencing with Integer Lengths問題
各々サイズ$(L_i)$の決まったジョブが$N$個あり、それを$m$個のマシンに割り当てるときに、マシン$\alpha$に割り当てられたジョブのサイズの合計$\displaystyle M_\alpha = \sum_{i \in alpha} L_i$の最大値$max(M_\alpha)$が最小になるような割り当て方を求める問題をJob Sequencing with Integer Lengths問題といいます($L_i$は正整数とします)。0番目のマシンのジョブサイズが最大になるとして一般性を失わないので、以下では$max(M_\alpha) = M_0$とします。また、その最大値をとる$M_0$と他の$M_\alpha$との差$M_0 - M_\alpha$を$n$とします。

# ハミルトニアン
次の論文を参考にすると、ハミルトニアンは以下のようになります。  
https://arxiv.org/abs/1302.5843  
$\displaystyle H = H_A + H_B$  
$\displaystyle H_A = A \sum_{i=1}^N \left( 1 - \sum_\alpha x_{i,\alpha} \right)^2 + A\sum_{\alpha=1}^m \left( \sum_{n=1}^M ny_{n,\alpha} + \sum_i L_i \left( x_{i,\alpha} - x_{i,1}\right) \right)^2$  
$\displaystyle H_B = B \sum_i L_ix_i$  
$x_{i, \alpha}$は求めたい解そのものです。各ジョブ$i$がマシン$\alpha$に割り当てられるときこの値は1を取ります。$H_A$の第一項は$x_{i, \alpha}$が各$i$についてひとつの$\alpha$で1となることを保証する項です。$y_{n, \alpha}$は補助変数です。各マシン$\alpha$について、$M_0 - M_\alpha = n$となるときこの値は1を取ります。$H_A$の第二項は各マシン$\alpha$について割り当てられたジョブのサイズを$x_{i, \alpha}$と$y_{n, \alpha}$で表現したときにそれらが等しくなることを保証する項となります。  

# ハミルトニアンを少し工夫してみる
上記のハミルトニアンのままで実装を行いましたが、良い解が得られないため、以下の通り改変を行ってみました。  
①$H_A$の各項はスケールが違うので係数を$A_1,A_2$に分解し、それぞれ設定できるようにしました。  
②$y_{n,\alpha}$は各$\alpha$についてひとつの$n$で1を取ると考えて$H_A$に$\displaystyle A_1\sum_{\alpha}\left( 1 - \sum_n y_{n,\alpha} \right)^2$の項を加えました。  
特に、①の変更を入れたことで解が安定しました。これらの変更を加えた$H_A$を以下のように変形しておきます。

$\displaystyle H_A = A_1\sum_i \left( 1 - \sum_\alpha x_{i,\alpha} \right)^2 + A_1\sum_{\alpha}\left( 1 - \sum_n y_{n,\alpha} \right)^2 + A_2\sum_\alpha \left( \sum_n ny_{n,\alpha} + \sum_i L_i \left( x_{i,\alpha} - x_{i,1}\right) \right)^2$  
$\displaystyle = A_1\sum_i \left( -2 \sum_\alpha x_{i,\alpha} + \left( \sum_\alpha x_{i,\alpha} \right)^2 \right) + A_1\sum_\alpha \left( -2 \sum_n y_{n,\alpha} + \left( \sum_n y_{n,\alpha} \right)^2 \right)$  
$\displaystyle + A_2\sum_\alpha \left( \left( \sum_n ny_{n,\alpha} \right)^2 + 2\left( \sum_n ny_{n,\alpha} \right)\left(\sum_i L_i \left( x_{i,\alpha} - x_{i,1}\right)\right) +\left(\sum_i L_i \left( x_{i,\alpha} - x_{i,1}\right)\right)^2 \right) + Const.$
$\displaystyle = A_1\sum_i \sum_\alpha \left( -x_{i,\alpha}^2 + \sum_{\beta \left( \gt \alpha \right)} 2x_{i,\alpha}x_{i, \beta} \right) + A_1\sum_\alpha \sum_n \left( -y_{n,\alpha}^2 + \sum_{m \left( \gt n \right) } 2y_{n,\alpha}y_{m, \alpha} \right)　+ A_2\sum_\alpha \sum_n \left( n^2y_{n, \alpha}^2 + \sum_{m \left( \gt n \right) } 2nmy_{n,\alpha}y_{m, \alpha} \right) $  
$\displaystyle + A_2\sum_\alpha \sum_i \left( \left( \sum_n 2nL_i y_{n,\alpha} \left( x_{i,\alpha} - x_{i,1}\right) \right) + L_i^2 \left( x_{i,\alpha} - x_{i,1}\right)^2 + \sum_{j \left( \gt i \right) } 2L_iL_j \left( x_{i,\alpha} - x_{i,1}\right) \left( x_{j,\alpha} - x_{j,1}\right) \right) + Const.$  
$\displaystyle =\sum_\alpha \sum_i \left( - A_1x_{i,\alpha}^2 +  A_2L_i^2 \left( x_{i,\alpha} - x_{i,1}\right)^2 + \sum_{\beta \left( \gt \alpha \right) }  2A_1x_{i,\alpha}x_{i, \beta} + \sum_{j \left( \gt i \right) } 2A_2L_iL_j \left( x_{i,\alpha} - x_{i,1}\right) \left( x_{j,\alpha} - x_{j,1}\right)+ \sum_n 2A_2nL_i y_{n,\alpha} \left( x_{i,\alpha} - x_{i,1}\right) \right)$  
$\displaystyle + \sum_\alpha \sum_n \left( \left( -A_1 + A_2n^2 \right) y_{n, \alpha}^2 + \sum_{m \left( \gt n \right) } 2\left(A_1+ A_2nm \right) y_{n,\alpha}y_{m, \alpha} \right) + Const.$  

# QUBOを計算して問題を解く
QUBO行列を計算するクラスを作成します。

In [1]:
!pip install blueqat

     |████████████████████████████████| 51kB 1.6MB/s 


In [0]:
import blueqat.opt as wq
import numpy as np

class Qubo():
    def __init__(self, jobs, n_machine, max_delta, A1, A2, B):
        self.__jobs = jobs
        self.__n_jobs = len(jobs)
        self.__n_machine = n_machine
        self.__max_delta = max_delta
        self.__A1 = A1
        self.__A2 = A2
        self.__B = B
        self.__index_offset = self.__n_jobs * self.__n_machine

    def __calc_sum_alpha_n_m(self, qubo, alpha, n):
        A1 = self.__A1
        A2 = self.__A2
        for m in range(n + 1, self.__max_delta + 1):
            v_n_alpha = self.__index_offset + alpha * self.__max_delta + n - 1
            v_m_alpha = self.__index_offset + alpha * self.__max_delta + m - 1
            qubo[v_n_alpha][v_m_alpha] += 2 * (A2 * n * m + A1)

    def __calc_sum_alpha_n(self, qubo, alpha):
        A1 = self.__A1
        A2 = self.__A2
        for n in range(1, self.__max_delta + 1):
            v_n_alpha = self.__index_offset + alpha * self.__max_delta + n - 1
            qubo[v_n_alpha][v_n_alpha] += (A2 * n ** 2 - A1)
            
            self.__calc_sum_alpha_n_m(qubo, alpha, n)

    def __calc_sum_alpha_i_beta(self, qubo, alpha, i):
        A1 = self.__A1
        for beta in range(alpha + 1, self.__n_machine):
            u_i_alpha = i * self.__n_machine + alpha
            u_i_beta = i * self.__n_machine + beta
            qubo[u_i_alpha][u_i_beta] += 2 * A1

    def __calc_sum_alpha_i_j(self, qubo, alpha, i):
        A2 = self.__A2
        Li = self.__jobs[i]
        for j in range(i + 1, self.__n_jobs):
            Lj = self.__jobs[j]
            u_i_alpha = i * self.__n_machine + alpha
            u_j_alpha = j * self.__n_machine + alpha
            u_i_0 = i * self.__n_machine 
            u_j_0 = j * self.__n_machine
            qubo[u_i_alpha][u_j_alpha] += 2 * A2 * Li * Lj
            qubo[u_i_alpha][u_j_0] += -2 * A2 * Li * Lj
            qubo[u_i_0][u_j_alpha] += -2 * A2 * Li * Lj
            qubo[u_i_0][u_j_0] += 2 * A2 * Li * Lj

    def __calc_sum_alpha_i_n(self, qubo, alpha, i):
        A2 = self.__A2
        Li = self.__jobs[i]
        u_i_alpha = i * self.__n_machine + alpha
        u_i_0 = i * self.__n_machine
        for n in range(1, self.__max_delta + 1):
            v_n_alpha = self.__index_offset + alpha * self.__max_delta + n - 1
            qubo[u_i_alpha][v_n_alpha] += 2 * A2 * n * Li
            qubo[u_i_0][v_n_alpha] += -2 * A2 * n * Li

    def __calc_sum_alpha_i(self,qubo, alpha):
        A1 = self.__A1
        A2 = self.__A2
        for i in range(self.__n_jobs):
            u_i_alpha = i * self.__n_machine + alpha
            u_i_0 = i * self.__n_machine
            Li = self.__jobs[i]
            qubo[u_i_alpha][u_i_alpha] += -A1 + A2 * Li ** 2
            qubo[u_i_0][u_i_0] += A2 * Li ** 2
            qubo[u_i_0][u_i_alpha] += -2 * A2 * Li ** 2

            self.__calc_sum_alpha_i_beta(qubo, alpha, i)
            self.__calc_sum_alpha_i_j(qubo, alpha, i)
            self.__calc_sum_alpha_i_n(qubo, alpha, i)

    def __calc_constraint_func(self,qubo):
        for alpha in range(self.__n_machine):
            self.__calc_sum_alpha_i(qubo, alpha)
            self.__calc_sum_alpha_n(qubo, alpha)

    def __calc_objective_func(self,qubo):
        B = self.__B
        for i in range(self.__n_jobs):
            u_i_0 = i * self.__n_machine
            Li = self.__jobs[i]
            qubo[u_i_0][u_i_0] += B * Li

    def __calc_qubo(self, qubo):
        self.__calc_constraint_func(qubo)
        self.__calc_objective_func(qubo)

    def get_qubo(self):
        size = self.__n_machine * (self.__n_jobs + self.__max_delta)
        qubo = np.zeros((size, size))
        self.__calc_qubo(qubo)
        return qubo

    def show_answer(self, solution):
        print(f"Solution is {solution}")
        assigned_job_sizes = np.zeros(self.__n_machine, dtype=int)
        for i in range(self.__n_jobs):
            assigned = False
            for alpha in range(self.__n_machine):
                u_i_alpha = i * n_machine + alpha
                if(solution[u_i_alpha] > 0):
                    print(f"Job{i} has been assigned to the machine{alpha}.")
                    assigned_job_sizes[alpha] += self.__jobs[i]
                    assigned = True
            if assigned == False:
                    print(f"Job{i} has not been assigned.")
        for alpha in range(self.__n_machine):
            print(f"Total size of jobs assigned to machine{alpha} is {assigned_job_sizes[alpha]}.")


問題を設定し、解きます。$A1, A2, B$のバランスは、それぞれの係数がかかっている項全体のスケールのバランスを考慮して決定しています。

In [4]:
jobs = [1,1,2,2,5,5,7] # the numbers are lengths(Li) of jobs
n_machine = 3
max_delta = 7 # permissive maximum delta of M1 - Malpha. select by yourself.
A1 = 1
A2 = (A1 / max(jobs) ** 2) * 0.9
B = (A1 / max(jobs)) * 0.5
qubo = Qubo(jobs, n_machine, max_delta, A1, A2, B)
annealer = wq.opt()
annealer.qubo = qubo.get_qubo()
for _ in range(10):
    solution = annealer.sa()
    qubo.show_answer(solution)
    print()

Solution is [1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
Job0 has been assigned to the machine0.
Job1 has been assigned to the machine2.
Job2 has been assigned to the machine0.
Job3 has been assigned to the machine2.
Job4 has been assigned to the machine2.
Job5 has been assigned to the machine0.
Job6 has been assigned to the machine1.
Total size of jobs assigned to machine0 is 8.
Total size of jobs assigned to machine1 is 7.
Total size of jobs assigned to machine2 is 8.

Solution is [0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
Job0 has been assigned to the machine2.
Job1 has been assigned to the machine1.
Job2 has been assigned to the machine0.
Job3 has been assigned to the machine2.
Job4 has been assigned to the machine1.
Job5 has been assigned to the machine2.
Job6 has been assigned to the machine0.
Total size of jobs assi

補助変数まで正確に答えを出すのは難しいですが、答えはそれなりに得られています。